In [5]:
import pandas as pd
import numpy as np
import tqdm

In [6]:
from reader import get_comments

x_train, y_train, x_test, y_test = get_comments()

In [7]:
print(y_train.sum(axis=0))

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64


In [9]:
import pickle
with open('vectorizer.pickle', 'rb') as f:
    vectorizer = pickle.load(f)
with open('results.pickle', 'rb') as f:
    results = pickle.load(f)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity 

In [37]:
from max_holder import MaxHolder

def find_n_closest(vect, n=1):
    mh = MaxHolder[int](n)
    for i, r in tqdm.tqdm(enumerate(results)):
        a = cosine_similarity(vect, r)
        mh.process(key=a, item=i)
    scores, indices = mh.contents()
    return indices

In [ ]:
res = list(find_n_closest(results[58], n=5))

In [ ]:
y_train.iloc[res]

In [ ]:
print(x[76])

In [43]:
res = list(find_n_closest(results[58], 10))
print(res)

100%|██████████| 159571/159571 [01:05<00:00, 2445.97it/s]


In [11]:
n = results[0].shape[1]
print(n)

136628


In [13]:
toxicity = [r for r in y_train.toxic]

In [15]:
A, a_cnt = np.zeros((1,n)), 0
B, b_cnt = np.zeros((1,n)), 0

for i,t in enumerate(tqdm.tqdm(results)):
    toxic = y_train.iloc[i].toxic == 1
    if toxic:
        A += t
        a_cnt += 1
    else:
        B += t
        b_cnt += 1

 18%|█▊        | 28727/159571 [00:12<00:58, 2244.22it/s]


KeyboardInterrupt: 

In [52]:
A_avg = A/a_cnt
B_avg = B/b_cnt

[[2.55639177e-05 0.00000000e+00 0.00000000e+00 1.38396662e-05
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
[[1.94193113e-05 1.35663305e-06 7.30122888e-07 0.00000000e+00
  3.02417438e-06 3.80110656e-06 5.28909271e-07 9.92100724e-07
  3.95914108e-07 1.50602469e-06 3.02339076e-06 6.39901272e-07]]


In [57]:
print(A_avg[0,:12])
print(B_avg[0,:12])


9970


In [69]:
cnt = 0
for i in range(n):
    if A_avg[0,i] > 5*B_avg[0,i]:
        cnt += 1
print(cnt)


ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [23]:
from _collections_abc import Sequence
class DataGenerator(Sequence):
    def __init__(self, dataset, batch_size=16, dim=(1), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.dataset = dataset
        self.shuffle = shuffle
        self.indexes = dataset.index
        self.on_epoch_end()
 
    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(len(self.dataset) / self.batch_size)
 
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        start = index*self.batch_size
        stop = (index+1)*self.batch_size
 
        data = [x.todense() for x in self.dataset[start:stop]]
        return data
 
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(dataset))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [25]:
training_gen = DataGenerator(results)

NameError: name 'dataset' is not defined

In [17]:
import tensorflow as tf

In [18]:
from keras.models import Sequential
from keras.layers import Dense

In [19]:
model = Sequential()
model.add(Dense(1000, input_dim=n, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [20]:
model.fit(results, toxicity, epochs=2, batch_size=32)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'scipy.sparse.csr.csr_matrix'>"}), (<class 'list'> containing values of types {"<class 'int'>"})